In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    #time.sleep(0.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    #time.sleep(0.5)    
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    #time.sleep(0.5)
    #driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

In [2]:
urls = pd.read_csv('mlb_stats_urls.csv', encoding='utf-8')

for year in range(2013, 2015):
    for link in urls['link']:
        df_list = []
        for page in range(1, 4):
            url = link + str(year) + '?page=' + str(page) + '&playerPool=ALL'
            driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
            html = driver.get(url)
            sp = soup(driver.page_source, 'lxml')
            column_name = get_column_name()
            try:
                #while True:
                sp = soup(driver.page_source, 'lxml')
                print('page start')
                time.sleep(1)
                name = get_player_name()
                time.sleep(1)
                position = get_player_position()
                time.sleep(1)
                current_data = get_data()
                time.sleep(1)
                for i in range(len(name)):
                    row = []
                    row.append(name[i])
                    row.append(position[i])
                    row.extend(current_data[i*32:i*32+32])
                    df_list.append(row)
                time.sleep(1)
                #click_next_page_button()
            except:
                print('This is the last page :)')
        df = pd.DataFrame(df_list, columns=column_name)
        df.to_csv(f'mlb_stats/{year}/{link[26:-1]}_{year}.csv', index=False, encoding='utf-8')
        print(df)

C:\Users\user\AppData\Local\Temp\ipykernel_29032\4135522725.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
              PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ...    3B  \
0        John Hester        C  LAA    1    0    1    0   0  0   0  ...     0   
1         Mike Trout       CF  LAA  157  589  109  190  39  9  27  ...    39   
2       Kole Calhoun       RF  LAA   58  195   29   55   7  2   8  ...    55   
3     Howie Kendrick       2B  LAA  122  478   55  142  21  4  13  ...    55   
4      Albert Pujols       DH  LAA   99  391   49  101  19  0  17  ...   391   
5      Efren Navarro       1B  LAA    4    4    0    1   0  0   0  ...     4   
6        Mark Trumbo       1B  LAA  159  620   85  145  30  2  34  ...   LAA   
7      Josh Hamilton       RF  LAA  151  576   73  144  32  5  21  ...  .747   
8     Chris Iannetta        C  LAA  115  325   40   73  15  0  11  ...  .432   
9        Grant Green       2B  LAA   40  125   16   35   8  1   1  ...  .358   
10       Hank Conger        C  LAA   92  233   23   58  13  1   7  ...  .280   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_29032\4135522725.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0        Jason Castro        C  HOU  120  435  63  120  35  1  18  ...  0.95   
1        Chris Carter       1B  HOU  148  506  64  113  24  2  29  ...  0.65   
2       Lucas Harrell        P  HOU   37    4   0    1   0  0   0  ...  1.00   
3         Rick Ankiel       RF  HOU   25   62   6   12   3  0   5  ...  0.36   
4       Brett Wallace       1B  HOU   79  262  35   58  14  1  13  ...  1.00   
5           L.J. Hoes       RF  HOU   46  167  24   48   7  2   1  ...  2.31   
6     Robbie Grossman       LF  HOU   63  257  29   69  14  0   4  ...  1.07   
7      Justin Maxwell       CF  HOU   40  137  21   33  10  2   2  ...  1.03   
8      Matt Dominguez       3B  HOU  152  543  56  131  25  0  21  ...  0.95   
9         Jose Altuve       2B  HOU  152  626  64  177  31  2   5  ...  1.24   
10        Carlos Pena       1B  HOU   85  277  38   58  13  1   8  ...  1.67   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_29032\4135522725.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


...

page start
page start
page start
              PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0         Mike Trout       CF  LAA  157  602  115  173  39  9  36  ...  0.58   
1      Albert Pujols       1B  LAA  159  633   89  172  37  1  28  ...  1.15   
2       Kole Calhoun       RF  LAA  127  493   90  134  31  3  17  ...  1.06   
3     Chris Iannetta        C  LAA  108  306   41   77  22  0   7  ...  0.79   
4     Gordon Beckham       3B  LAA   26   56   10   15   3  0   2  ...  1.00   
5      Josh Hamilton       LF  LAA   89  338   43   89  21  0  10  ...  0.91   
6     Howie Kendrick       2B  LAA  157  617   85  181  33  5   7  ...  2.09   
7          C.J. Cron       DH  LAA   79  242   28   62  12  1  11  ...  0.70   
8       David Freese       3B  LAA  134  462   53  120  25  1  10  ...  1.30   
9        Erick Aybar       SS  LAA  156  589   77  164  30  4   7  ...  1.25   
10    Collin Cowgill       RF  LAA  106  260   37   65  10  1   5  ...  1.05   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_29032\4135522725.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0       Samuel Deduno        P  HOU    5    1   0    1   1  0   0  ...  -.--   
1         Jose Altuve       2B  HOU  158  660  85  225  47  3   7  ...  1.05   
2     George Springer       RF  HOU   78  295  45   68   8  1  20  ...  1.02   
3          Max Stassi        C  HOU    7   20   2    7   2  0   0  ...  0.75   
4        Chris Carter       DH  HOU  145  507  68  115  21  1  37  ...  0.47   
5       Dexter Fowler       CF  HOU  116  434  61  120  21  4   8  ...  0.98   
6   Enrique Hernandez       CF  HOU   24   81  10   23   4  2   1  ...  0.74   
7     Marwin Gonzalez       SS  HOU  103  285  33   79  15  1   6  ...  1.49   
8      Gregorio Petit       SS  HOU   37   97  14   27   8  0   2  ...  1.05   
9     Carlos Corporan        C  HOU   55  170  22   40   6  0   6  ...  0.88   
10    Robbie Grossman       LF  HOU  103  360  42   84  14  2   6  ...  1.05   
11     

C:\Users\user\AppData\Local\Temp\ipykernel_29032\4135522725.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
This is the last page :)


In [ ]:
driver.quit()